In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.unet_enhancement as unet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [20]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 8192)
model = unet.Model(partitioned_x, channels_interval = 36)

Tensor("strided_slice_3:0", shape=(?, 4096, 36), dtype=float32)
Tensor("strided_slice_4:0", shape=(?, 2048, 72), dtype=float32)
Tensor("strided_slice_5:0", shape=(?, 1024, 108), dtype=float32)
Tensor("strided_slice_6:0", shape=(?, 512, 144), dtype=float32)
Tensor("strided_slice_7:0", shape=(?, 256, 180), dtype=float32)
Tensor("strided_slice_8:0", shape=(?, 128, 216), dtype=float32)
Tensor("strided_slice_9:0", shape=(?, 64, 252), dtype=float32)
Tensor("strided_slice_10:0", shape=(?, 32, 288), dtype=float32)
Tensor("strided_slice_11:0", shape=(?, 16, 324), dtype=float32)
Tensor("strided_slice_12:0", shape=(?, 8, 360), dtype=float32)
Tensor("strided_slice_13:0", shape=(?, 4, 396), dtype=float32)
Tensor("strided_slice_14:0", shape=(?, 2, 432), dtype=float32)
Tensor("sequential_12/leaky_re_lu_12/LeakyRelu:0", shape=(?, 2, 432), dtype=float32)
Tensor("sequential_13/leaky_re_lu_13/LeakyRelu:0", shape=(?, 4, 432), dtype=float32)
Tensor("sequential_14/leaky_re_lu_14/LeakyRelu:0", shape=(?, 8, 3

In [21]:
model.logits

<tf.Tensor 'sequential_25/activation/Tanh:0' shape=(?, 8192, 1) dtype=float32>

In [22]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_40:0' shape=(?, 1) dtype=float32>

In [23]:
snr = enhancement.loss.snr(model.logits, partitioned_x)
sdr = enhancement.loss.sdr(model.logits, partitioned_x)

In [24]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [25]:
import numpy as np

In [26]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [27]:
y_, snr_, sdr_ = sess.run([logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(258867, 1)

In [28]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [29]:
snr_, sdr_

((0.49020642, -18.220104), 6.6332905e-07)

In [30]:
y

array([-0.00570545, -0.01051882, -0.01370371, ...,  0.0140948 ,
        0.01236383,  0.01233889])

In [31]:
y_[:,0]

array([-0.3180574 , -0.23235135, -0.01387705, ...,  0.5146223 ,
        0.46609902,  0.3781421 ], dtype=float32)

In [32]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [33]:
!ls -lh test

total 179296
-rw-r--r--  1 huseinzolkepli  staff    77B Jan 17 18:21 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    87M Jan 17 18:21 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   6.1K Jan 17 18:21 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   560K Jan 17 18:21 model.ckpt.meta


In [34]:
!rm -rf test